In [3]:
import csv
import cv2
import numpy as np
import matplotlib.image as mpimg
import glob
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Lambda,Conv2D,MaxPooling2D,Convolution2D
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda
from sklearn.utils import shuffle
from sklearn import model_selection
from keras.regularizers import l2
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam
from keras.models import Model
import matplotlib.pyplot as plt


/home/wuying/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
#def load data


image_paths=[]
angles=[]

img_path_prepend='/home/wuying/Desktop/IMG'

with open('./driving_log.csv') as f :
    driving_data = list(csv.reader(f, skipinitialspace=True, delimiter=',', quoting=csv.QUOTE_NONE))
    train_samples, validation_samples = model_selection.train_test_split(driving_data, test_size=0.2)   
    # Gather data - image paths and angles for center, left, right cameras in each row
    for row in driving_data[1:]:
        # skip it if ~0 speed - not representative of driving behavior
        if float(row[6]) < 0.1 :
            continue
        # get center image path and angle
        image_paths.append(img_path_prepend + '/'+row[0].split('/')[-1])
        angles.append(float(row[3]))
        # get left image path and angle
        image_paths.append(img_path_prepend +  '/'+row[1].split('/')[-1])
        angles.append(float(row[3])+0.25)
        # get left image path and angle
        image_paths.append(img_path_prepend +  '/'+row[2].split('/')[-1])
        angles.append(float(row[3])-0.25)

image_paths = np.array(image_paths)
angles = np.array(angles)
print(train_samples)
print(image_paths)
print(angles)

[['/home/wuying/Desktop/IMG/center_2018_04_13_19_31_22_738.jpg', '/home/wuying/Desktop/IMG/left_2018_04_13_19_31_22_738.jpg', '/home/wuying/Desktop/IMG/right_2018_04_13_19_31_22_738.jpg', '0', '0', '0', '18.43039'], ['/home/wuying/Desktop/IMG/center_2018_04_13_19_52_40_019.jpg', '/home/wuying/Desktop/IMG/left_2018_04_13_19_52_40_019.jpg', '/home/wuying/Desktop/IMG/right_2018_04_13_19_52_40_019.jpg', '0', '0.288168', '0', '6.455251'], ['/home/wuying/Desktop/IMG/center_2018_04_13_19_48_48_422.jpg', '/home/wuying/Desktop/IMG/left_2018_04_13_19_48_48_422.jpg', '/home/wuying/Desktop/IMG/right_2018_04_13_19_48_48_422.jpg', '0', '0', '0', '7.531535'], ['/home/wuying/Desktop/IMG/center_2018_04_13_19_56_03_680.jpg', '/home/wuying/Desktop/IMG/left_2018_04_13_19_56_03_680.jpg', '/home/wuying/Desktop/IMG/right_2018_04_13_19_56_03_680.jpg', '0', '0', '0', '15.49592'], ['/home/wuying/Desktop/IMG/center_2018_04_13_19_52_06_229.jpg', '/home/wuying/Desktop/IMG/left_2018_04_13_19_52_06_229.jpg', '/home/

In [9]:
#preprocessing

from keras.models import Sequential
from keras.layers import Flatten, Dense,Lambda


def cropped_image(image):
    cropped_image=image[45:130,:,:]
    resized_image=cv2.resize(cropped_image,(66,200))
    return resized_image

def brightness_image(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    image[:,:,2] = image[:,:,2]*random_bright
    image = cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
    return image
    
def proprcessing_image(image):
    image=cropped_image(image)
    image=brightness_image(image)
    return image

In [10]:
def generate_training_data_for_visualization(image_paths, angles,validation_flag=False):
    '''
    method for loading, processing, and distorting images
    if 'validation_flag' is true the image is not distorted
    '''
    x_train = []
    train = []
    image_paths, angles = shuffle(image_paths, angles)
    for i in range(0,len(angles),10):
        image = cv2.imread(image_paths[i])
        angle = angles[i]
        image=proprcessing_image(image)
      # if not validation_flag:
           # image, angle = random_distort(img, angle)
        x.append(image)
        y.append(angle)
        
    yield shuffle(x_train, y_train)



#x_train,y_train=generate_training_data_for_visualization(image_paths,angles)
train_generator=generate_training_data_for_visualization()
TRAIN_GENERATOR = generator(TRAIN_SAMPLES, batch_size=32)
VALIDATION_GENERATOR = generator(VALIDATION_SAMPLES, batch_size=32
#print(x.shape)
#print(y_train)
#print(validation_generator)

NameError: name 'x' is not defined

In [ ]:
number_of_epochs = 5
number_of_samples_per_epoch = len(x_train)
number_of_validation_samples = len(validation_generator)
learning_rate = 1e-4
activation_relu = 'relu'

# Our model is based on NVIDIA's "End to End Learning for Self-Driving Cars" paper
# Source:  https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
model = Sequential()

model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=(200, 66, 3)))

# starts with five convolutional and maxpooling layers
model.add(Convolution2D(24, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(activation_relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(activation_relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
model.add(Activation(activation_relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(activation_relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
model.add(Activation(activation_relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Flatten())

# Next, five fully connected layers
model.add(Dense(1164))
model.add(Activation(activation_relu))

model.add(Dense(100))
model.add(Activation(activation_relu))

model.add(Dense(50))
model.add(Activation(activation_relu))

model.add(Dense(10))
model.add(Activation(activation_relu))

model.add(Dense(1))

model.summary()

model.compile(optimizer=Adam(learning_rate), loss="mse", )

# create two generators for training and validation
history = model.fit_generator(X_train,y_train,
                              steps_per_epoch=number_of_samples_per_epoch,
                              epochs=5,
                              validation_data=X_valid,
                              nb_val_samples=number_of_validation_samples,
                              verbose=1)

# finally save our model and weights
model.save('/home/wuying/Desktop/model.h5')

#define model
number_of_samples_per_epoch=x_train.shape[0]
number_of_validation_samples=x_valid.shape[0]
print(number_of_validation_samples)

def Model():
    #input 66*200*3
    model=Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.0,input_shape=(200,66,3)))
              
    
    # Add three 5x5 convolution layers (output depth 24, 36, and 48), each with 2x2 stride
    model.add(Conv2D(24, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(36, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(48, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    
    # Add two 3x3 convolution layers (output depth 64, and 64)
    model.add(Conv2D(64, 3, 3, border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Conv2D(64, 3, 3, border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    
    # Add three fully connected layers (depth 100, 50, 10), tanh activation (and dropouts)
    model.add(Dense(100, W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Dense(50, W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Dense(10, W_regularizer=l2(0.001)))
    model.add(ELU())
    
    # Add a fully connected output layer
    model.add(Dense(1))
 # Compile and train the model
    model.compile(optimizer=Adam(lr=1e-4), loss='mse',metrics=['accuracy'])

    return model
model=Model()
history_object =model.fit_generator(validation_generator,
                                    steps_per_epoch=10,
                                    validation_data = validation_generator,
                                    validation_steps = number_of_validation_samples, 
                                    epochs=5, 
                                    verbose=1,
                                     )
model.save('/home/wuying/Desktop/model.h5')
                                    
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
#plt.plot(history_object.history['loss'])
#plt.plot(history_object.history['val_loss'])
#plt.title('model mean squared error loss')
#plt.ylabel('mean squared error loss')
#plt.xlabel('epoch')
#plt.legend(['training set', 'validation set'], loc='upper right')
#plt.show()
print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])